## Junta tablas

In [8]:
import os
import pandas as pd
import re
from datetime import datetime

In [9]:
def read_and_join_csvs_by_date(fecha, folder_path='1_datos_raw'):
    """
    Función que toma una fecha en formato 'yyyymmdd' y busca los archivos CSV 
    en el folder_path que comiencen con esa fecha, añadiendo la fecha como columna
    y realizando un outer join entre ellos.

    Args:
    -(fecha) (str): Fecha en formato 'yyyymmdd'.
    - folder_path (str): Directorio donde se encuentran los archivos CSV. Por defecto, es el directorio actual.

    Returns:
    - pd.DataFrame: DataFrame resultante del outer join entre los CSV encontrados con la misma fecha,
                    incluyendo una columna "fecha".
    """
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Filtrar los archivos que comienzan con la fecha proporcionada
    csv_files = [f for f in files if f.startswith(fecha) and f.endswith('.csv')]

    # Verificar que haya más de un archivo para realizar el join
    if len(csv_files) > 1:
        # Leer los archivos con la misma fecha y añadir la columna "fecha"
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(folder_path, f))
            df['fecha'] =(fecha)  # Añadir la columna con la fecha
            dfs.append(df)

        # Hacer un join de todos los DataFrames de esa fecha usando outer join
        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, how='outer')  # Outer join para añadir registros no coincidentes
        
        return merged_df
    else:
        print(f"No se encontraron suficientes archivos con la fecha {fecha} para hacer un join.")
        return None

# Uso de la función
fecha = datetime.now().strftime("%Y%m%d")
#fecha = '20241114'  # Fecha en formato 'yyyymmdd'
df = read_and_join_csvs_by_date(fecha)
del df['Unnamed: 0']
df

,id,nombre,precio,estado,reservado,url,fecha
0,1073642029,iphone 15,"590,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-15-1073642029,20241127
1,1073757376,iphone 15 128gb negro,"599,00",good,False,https://es.wallapop.com/item/iphone-15-128gb-n...,20241127
2,1073789336,iphone 15 pro,"899,00",new,False,https://es.wallapop.com/item/iphone-15-pro-107...,20241127
3,1073756454,iphone 15 plus a estrenar,"800,00",new,False,https://es.wallapop.com/item/iphone-15-plus-a-...,20241127
4,1073765761,iphone 15 pro 128gb natural,"799,00",good,False,https://es.wallapop.com/item/iphone-15-pro-128...,20241127
...,...,...,...,...,...,...,...
767,1064214990,iphone 15 rosa 128 gb,"690,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-15-rosa-12...,20241127
768,1064076902,iphone 15 pro max,"1100,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-15-pro-max...,20241127
769,1066656872,iphone 15 pro max 800€,"850,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-15-pro-max...,20241127
770,1066507974,iphone 15 pro max 1tb titanio blanco,"1249,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-15-pro-max...,20241127


In [10]:
# Creo una columna en la que pongo que iphone es (14, 15 o 16..) y que tipo. 
# Gen
# Model
for index in range(len(df)):
    df['gen'] = 'vacio'
    df['modelo'] = 'modelo'


In [11]:
def asignar_gen(nombre):
    if '14' in nombre:
        return '14'
    elif '15' in nombre:
        return '15'
    elif '16' in nombre:
        return '16'
    return "n/a"

# Función para asignar el valor de la columna "modelo"
def asignar_modelo(nombre):
    if 'pro max' in nombre.lower():
        return 'pro max'
    elif 'pro' in nombre.lower():
        return 'pro'
    elif 'plus' in nombre.lower():
        return 'plus'
    else:
        return 'basic'

def asignar_memoria(nombre):
    if '128' in nombre.lower():
        return '128'
    elif '256' in nombre.lower():
        return '256'
    elif '512' in nombre.lower():
        return '512'
    elif '1tb' in nombre.lower():
        return '1tb'
    else:
        return 'n/a'

# Función para asignar el valor de la columna "bateria"
def asignar_bateria(nombre):
    # Buscar números entre 80 y 100 en el nombre
    bateria = re.findall(r'\b([8-9][0-9]|100)\b', nombre)
    if bateria:
        return int(bateria[0])  # Retorna el primer número que encuentra
    return "n/a"

# Aplicar las funciones al DataFrame
df['gen'] = df['nombre'].apply(asignar_gen)
df['mod'] = df['nombre'].apply(asignar_modelo)
df['memoria'] = df['nombre'].apply(asignar_memoria)
df['bateria'] = df['nombre'].apply(asignar_bateria)
orden_columnas = ['id','fecha', 'gen', 'mod', 'memoria', 'bateria','precio', 'estado', 'nombre',  'reservado', 'url']
df = df[orden_columnas]
df

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url
0,1073642029,20241127,15,basic,n/a,n/a,"590,00",as_good_as_new,iphone 15,False,https://es.wallapop.com/item/iphone-15-1073642029
1,1073757376,20241127,15,basic,128,n/a,"599,00",good,iphone 15 128gb negro,False,https://es.wallapop.com/item/iphone-15-128gb-n...
2,1073789336,20241127,15,pro,n/a,n/a,"899,00",new,iphone 15 pro,False,https://es.wallapop.com/item/iphone-15-pro-107...
3,1073756454,20241127,15,plus,n/a,n/a,"800,00",new,iphone 15 plus a estrenar,False,https://es.wallapop.com/item/iphone-15-plus-a-...
4,1073765761,20241127,15,pro,128,n/a,"799,00",good,iphone 15 pro 128gb natural,False,https://es.wallapop.com/item/iphone-15-pro-128...
...,...,...,...,...,...,...,...,...,...,...,...
767,1064214990,20241127,15,basic,128,n/a,"690,00",as_good_as_new,iphone 15 rosa 128 gb,False,https://es.wallapop.com/item/iphone-15-rosa-12...
768,1064076902,20241127,15,pro max,n/a,n/a,"1100,00",as_good_as_new,iphone 15 pro max,False,https://es.wallapop.com/item/iphone-15-pro-max...
769,1066656872,20241127,15,pro max,n/a,n/a,"850,00",as_good_as_new,iphone 15 pro max 800€,False,https://es.wallapop.com/item/iphone-15-pro-max...
770,1066507974,20241127,15,pro max,1tb,n/a,"1249,00",as_good_as_new,iphone 15 pro max 1tb titanio blanco,False,https://es.wallapop.com/item/iphone-15-pro-max...


In [12]:
carpeta = "2_datos_por_fecha"
nombre_archivo_pkl = carpeta + '/' + fecha + '.csv'
df.to_csv(nombre_archivo_pkl)
print(nombre_archivo_pkl)

2_datos_por_fecha/20241127.csv
